In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
import requests
import seaborn as sns

In [10]:
cd "C:\Users\Andre\Videos\mlclass-master\03_Validation"

C:\Users\Andre\Videos\mlclass-master\03_Validation


In [11]:
ls

 O volume na unidade C ‚ Andre_Santos
 O N£mero de S‚rie do Volume ‚ 384C-424E

 Pasta de C:\Users\Andre\Videos\mlclass-master\03_Validation

13/12/2020  18:40    <DIR>          .
13/12/2020  18:40    <DIR>          ..
13/12/2020  18:40    <DIR>          .ipynb_checkpoints
13/12/2020  18:40            29.517 03_Validation 2.0.ipynb
13/12/2020  18:31            36.379 03_Validation.ipynb
02/04/2020  16:53            51.833 abalone_app.csv
02/04/2020  16:53            46.265 abalone_app.xlsx
02/04/2020  16:53           160.978 abalone_dataset.csv
02/04/2020  16:53           137.879 abalone_dataset.xlsx
02/04/2020  16:53             3.780 README.md
               7 arquivo(s)        466.631 bytes
               3 pasta(s)   145.853.276.160 bytes dispon¡veis


In [20]:
data = pd.read_csv('abalone_dataset.csv')
data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
0,M,0.535,0.420,0.150,0.6995,0.2575,0.1530,0.2400,3
1,I,0.510,0.380,0.115,0.5155,0.2150,0.1135,0.1660,1
2,I,0.185,0.130,0.045,0.0290,0.0120,0.0075,0.0095,1
3,M,0.550,0.450,0.170,0.8100,0.3170,0.1570,0.2200,3
4,I,0.535,0.415,0.150,0.5765,0.3595,0.1350,0.2250,1


In [21]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
new_sex = encoder.fit_transform(data.sex)

data_with_encoded_sex = data.copy()
data_with_encoded_sex['sex'] = new_sex
data_with_encoded_sex.head(3)

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
0,2,0.535,0.42,0.150,0.6995,0.2575,0.1530,0.2400,3
1,1,0.510,0.38,0.115,0.5155,0.2150,0.1135,0.1660,1
2,1,0.185,0.13,0.045,0.0290,0.0120,0.0075,0.0095,1


<h1>Correlation between variables </h1>
</br>
Deleted variables: sex (low correlation), whole_weight,viscera_weight (high correlation with <strong>shell_weight</strong>)

In [22]:
data_with_encoded_sex.corr()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
sex,1.000000,-0.026521,-0.029353,-0.030344,-0.009325,0.012464,-0.018009,-0.023907,-0.040599
length,-0.026521,1.000000,0.987063,0.891287,0.924125,0.895178,0.901025,0.895045,0.582482
diameter,-0.029353,0.987063,1.000000,0.898236,0.925135,0.891759,0.898265,0.902889,0.597654
height,-0.030344,0.891287,0.898236,1.000000,0.883098,0.832339,0.859120,0.880513,0.607524
whole_weight,-0.009325,0.924125,0.925135,0.883098,1.000000,0.969027,0.966725,0.954062,0.580626
shucked_weight,0.012464,0.895178,0.891759,0.832339,0.969027,1.000000,0.930324,0.879352,0.491211
viscera_weight,-0.018009,0.901025,0.898265,0.859120,0.966725,0.930324,1.000000,0.905025,0.564292
shell_weight,-0.023907,0.895045,0.902889,0.880513,0.954062,0.879352,0.905025,1.000000,0.630489
type,-0.040599,0.582482,0.597654,0.607524,0.580626,0.491211,0.564292,0.630489,1.000000


<h1> Features Selection </h1>

In [25]:
cols = ['length','diameter','height','shucked_weight','shell_weight','type']
data = data[cols]
data.head(5)

,length,diameter,height,shucked_weight,shell_weight,type
0,0.535,0.420,0.150,0.2575,0.2400,3
1,0.510,0.380,0.115,0.2150,0.1660,1
2,0.185,0.130,0.045,0.0120,0.0095,1
3,0.550,0.450,0.170,0.3170,0.2200,3
4,0.535,0.415,0.150,0.3595,0.2250,1


In [26]:
data.type.value_counts()

1    1078
3    1051
2    1003
Name: type, dtype: int64

<h1>Data split</h1>

In [27]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(data.drop('type',axis=1),data.type,test_size=0.2,random_state=42)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(2505, 5) (627, 5)
(2505,) (627,)


<h1>Hyperparameters Selection</h1>

In [47]:
from sklearn.neighbors import KNeighborsClassifier

def get_neighbors(n):
  neighbor = []

  neighbor.append(n+2)
  neighbor.append(1 if n-2 < 1 else n-2)
  neighbor.append(n+1)
  neighbor.append(1 if n-1 < 1 else n-1)
  neighbor.append(np.random.randint(10,100))

  return neighbor

def hill_climbing(n,iter):

  best_score = 0
  best_n = n

  for i in tqdm(range(iter)):
    for _n in get_neighbors(n):

      knn = KNeighborsClassifier(n_neighbors=_n)
      knn.fit(x_train,y_train)
      current_score = knn.score(x_test,y_test)
    
      if best_score < current_score:
        best_score = current_score
        best_n = _n

    n = best_n

  return best_n,best_score

In [ ]:
_n = np.random.randint(1,5)
n,score = hill_climbing(_n,100)

In [50]:
print('Best_n = {}, Best_score= {}'.format(n,score))

Best_n = 93, Best_score= 0.6539074960127592


<h1>Define Model</h1>

In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier


#rfc = RandomForestClassifier(n_estimators=7,max_depth=8)
#rfc.fit(x_train,y_train)

knn = KNeighborsClassifier(n_neighbors=93)
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=93, p=2,
                     weights='uniform')

<h1>Model Valitation</h1>

In [52]:
knn.score(x_test,y_test)

0.6539074960127592

<h1>Model Predict</h1>

In [55]:
cols_test = ['length','diameter','height','shucked_weight','shell_weight']
data_test = pd.read_csv('abalone_app.csv')
data_test = data_test[cols_test]
data_test.head(5)

,length,diameter,height,shucked_weight,shell_weight
0,0.600,0.480,0.175,0.4125,0.4150
1,0.545,0.385,0.150,0.5425,0.2845
2,0.645,0.520,0.180,0.5775,0.3170
3,0.640,0.510,0.170,0.5670,0.4090
4,0.655,0.540,0.215,0.6950,0.4440


In [57]:
y_pred = knn.predict(data_test)
y_pred

array([3, 2, 2, ..., 3, 1, 2], dtype=int64)

In [63]:
np.savetxt('y_pred.txt', y_pred)

<h1>Submit predicted values</h1>

In [58]:
# Enviando previsões realizadas com o modelo para o servidor
URL = 'https://aydanomachado.com/mlclass/03_Validation.php'

#TODO Substituir pela sua chave aqui
DEV_KEY = "Andre Santos"

# json para ser enviado para o servidor
data = {'dev_key':DEV_KEY,
        'predictions':pd.Series(y_pred).to_json(orient='values')}

# Enviando requisição e salvando o objeto resposta
r = requests.post(url = URL, data = data)

# Extraindo e imprimindo o texto da resposta
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")

 - Resposta do servidor:
 {"status":"success","dev_key":"Andre Santos","accuracy":0.6478468899521531,"old_accuracy":0.61913875598086} 

